In [1]:
import sys
sys.path.append('../')
import env

In [2]:
import env
import sys
import pandas as pd
import numpy as np

In [3]:
read_dir_name = 'Origin'
write_dir_name = 'next_move_area_per_time'

is_remove_oririn = False

road_to_area = {}
df_base = {}
target_time = '3600'


def func_road_to_area(args):
    df = pd.read_csv(env.get_full_path('include_area_-1', args),
                     encoding='Shift_JISx0213')
    df = df.loc[:, ['road', 'area']]
    for row in np.asanyarray(df):
        road_to_area[row[0]] = float(row[1])
        # create_road_to_area(row[0], row[1])

    road_to_area[np.nan] = np.nan


def interpolate_time(time):
    time = int(time)
    times_list = [3600 * (i + 1) for i in range(6)]
    new_time = ''

    if 0 <= time <= times_list[0]:
        new_time = times_list[0]
    elif times_list[0] <= time <= times_list[1]:
        new_time = times_list[1]
    elif times_list[1] <= time <= times_list[2]:
        new_time = times_list[2]
    elif times_list[2] <= time <= times_list[3]:
        new_time = times_list[3]
    elif times_list[3] <= time <= times_list[4]:
        new_time = times_list[4]
    elif times_list[4] <= time <= times_list[5]:
        new_time = times_list[5]

    return str(new_time)


def create2d(args, array):
    df = np.zeros((env.MAX_AREA_COUNT(), 3))
    df = pd.DataFrame(df, columns=['time', 'area', 'people'])

    for _times in env.TIMES_LIST():
        array[args.dir][args.ratio][args.seed][args.csv][_times] = {}
        index = 0
        for _area in range(env.MAX_AREA_COUNT()):
            array[args.dir][args.ratio][args.seed][args.csv][_times][_area] = {}

            for _area2 in range(env.MAX_AREA_COUNT()):
                df.loc[index, ['time', 'area']] = [_times, _area2]
                index += 1
            array[args.dir][args.ratio][args.seed][args.csv][_times][_area] = df.copy()


def count_per_time(args, df, _times, _area):
    for _, row in df.iterrows():
        row = list(row)
        row = row[3:]

        for i in range(len(row)):
            if (len(row) <= i + 1) or (row[i+1] == ' '):
                break

            if (type(row[i]) is str) and ('(census)' in row[i]):
                _time = row[i].split('@')[1]
                _time = interpolate_time(_time)
                road_name = row[i].split('(census)')[0]
                _area = road_to_area[road_name]
                next_area = road_to_area[row[i + 1].split('(census)')[0]]

                df_tmp = df_base[args.dir][args.ratio][args.seed][args.csv][_times][_area]
                df_tmp.loc[df_tmp['area'] == next_area, ['people']] += 1


def main(args):
    df = pd.read_csv(env.get_full_path(read_dir_name, args),
                     names=env.get_col_names(),
                     encoding='Shift_JISx0213')

    for _area in env.get_area_list():
        count_per_time(args, df, target_time, _area)
#         df_base[args.dir][args.ratio][args.seed][args.csv][target_time][int(_area)].to_csv(
#             env.get_full_path(write_dir_name, args, any=target_time + '_' + _area), index=False)

In [5]:
env.for_default(func_road_to_area, csv=['census'])
env.for_default_init(create2d, df_base, ['od'])

In [ ]:
env.for_default(main, csv=['od'])